# **EXPANSE**

EXPANSE (EX-tended Pixel ANalysis of SEDs) is a Python package for spatially resolved SED fitting and analysis. This notebook demonstrates the basics of how to create an EXPANSE ResolvedGalaxy object, load data, and fit the SEDs of individual pixels.

I assume if you have a copy of this notebook that you have cloned the EXPANSE repository. If not, you can install it using pip: - run pip install . in the root directory of the repository. Add -e if you want to install it in editable mode.


Firstly we will import some neccessary packages.

In [ ]:
from EXPANSE import ResolvedGalaxy
import numpy as np
from EXPANSE.utils import PhotometryBandInfo, FieldInfo
from astropy.coordinates import SkyCoord
import astropy.units as u

There are two ways to create a new ResolvedGalaxy object. The first is to create it directly from a galfind Galaxy object, which is a class from the [galfind package](https://galfind.readthedocs.io/en/latest/). The second is to create it from scratch, which is what we will do here.

To that end, we have imported PhotometryBandInfo and FieldInfo from the EXPANSE.utils directory. These are helper classes designed to make it easier to pass the information about the photometry bands and fields to the ResolvedGalaxy object.

Firstly we will define some basic information about the galaxy. We will give it a ID, and a survey or field name, and a redshift. We will define a position using an astropy SkyCoord object. We will list the available photometric bands by the filter designation, but this is primarily for our convenience at this stage.

Our test galaxy is in the CEERS field, and we will incorporate both the HST and NIRCam imaging data.

In [ ]:
galaxy_id = 1
survey = 'CEERS'
redshift = 2
ra = 214.8 # deg
dec = 52.7 # deg
cutout_size = 'auto' # pixels or auto to base on segmentation map
 
position = SkyCoord(ra = ra * u.deg, dec = dec * u.deg)

hst_bands = ['F606W', 'F814W']
nircam_bands = ['F115W', 'F150W', 'F200W', 'F277W', 'F356W', 'F410M', 'F444W']


# Position of the galaxy.

Now we will initialize some PhotometryBandInfo objects. The goal is to have one photometry band object for each filter in the survey, which links together the band name, image file, image file, segmentation map (if it exists) and other information.

The path provided can either be a direct path to the image, or to the parent directory of the image. If the path is to the parent directory, the code will search for the image file in that directory, and will print the file it finds. You can link one path to another by e.g. setting err_path = 'im_folder' below, which will search for the error image in the same directory as the main image. If you set err_path = None, the code will not search for an error image. If you set err_path = 'im', the code will search for an ERR extension in the same file as the main image.

The code will attempt to determine the zeropoint, image unit and pixel scale of the image from the header, but you can override this by setting the values manually.

Pay careful attention to the print statements, as they will tell you if the code is unable to find the image file, or if it is unable to determine the zeropoint, image unit or pixel scale. The code may make an assumption about the naming convention or header information of the image file, and if this assumption is incorrect, you will need to set the values manually.

Note that the below code is setup to load in my own data, and you will need to change the paths to the image files to match your own data.

In [ ]:
galaxy_info = []
# Add HST bands
for band in hst_bands:
    band_info = PhotometryBandInfo(band_name=band, survey=survey,
                                image_path = f'/raid/scratch/data/hst/{survey}/ACS_WFC/30mas/',
                                wht_path = None,
                                err_path = 'im_folder',
                                seg_path = f'/raid/scratch/work/austind/GALFIND_WORK/SExtractor/ACS_WFC/v9/{survey}/{survey}_{band.replace("F", "f")}_{band.replace("F", "f")}_sel_cat_v9_seg.fits',
    )
    galaxy_info.append(band_info)

# Now add NIRCam bands
for band in nircam_bands:
    band_info = PhotometryBandInfo(band_name=band, survey=survey,
                                    image_path = f'/raid/scratch/data/jwst/{survey}/mosaic_1084_wisptemp2_whtfix',
                                    wht_path = 'im',
                                    err_path = 'im',
                                    seg_path = f'/raid/scratch/work/austind/GALFIND_WORK/SExtractor/NIRCam/v9/{survey}/{survey}_{band}_{band}_sel_cat_v9_seg.fits',
    )

    galaxy_info.append(band_info)



Now we combine these PhotometryBandInfo objects into a list, and create a FieldInfo object. This object contains the information about the field, including the photometry bands, the segmentation map, and the pixel scale of the image.

In [ ]:
# Make a FieldInfo object
field_info = FieldInfo(galaxy_info)

We can then create a ResolvedGalaxy object, which will create the galaxy cutouts and store all provided information in a portable HDF5 file. This file can be loaded later to continue the analysis.

There are a number of other arguments you may wish to set. These are:

- **already_psf_matched**: If the PSF has already been matched between the bands, set this to True. If not, the code will attempt to match the PSF WebbPSF or provided PSF models (coming soon).

- **forced_phot_band**: If you want to perform forced photometry on a specific band, set this to the filter name (or list for an inverse variance weighted combination of bands).

- **dont_psf_match_bands**: If you do not want to match the PSF between certain bands, set this to a list of the filter names. This is useful if the PSF matching is not working well for certain bands.


In [ ]:
galaxy = ResolvedGalaxy.init_from_basics(galaxy_id = galaxy,
                                        field_info = field_info, 
                                        survey = survey,
                                        cutout_size=cutout_size,
                                        sky_coord=skycoord, 
                                        redshift = redshift)

We can use the returned object directly, or load it from the HDF5 file. The HDF5 filenames generally have the format 'fieldname_galaxyid.hdf5', but you can specify a different filename if you wish.

In [ ]:
galaxy = ResolvedGalaxy.init_from_h5('CEERS_1.h5')

There are many methods available on a ResolvedGalaxy, but we will go over the most important ones here.

Firstly, we can plot the galaxy cutouts. This will show the galaxy cutouts for each band. 

In [ ]:
galaxy.plot_cutouts

galaxy.plot_err_stamps

galaxy.plot_seg_stamps

galaxy.plot_image_stamps

Plotting an RGB

In [ ]:
galaxy.plot_lupton_rgb

Saving info to .h5. Generally this will happen automatically, but if you want to force a save, you can call the dump_to_h5 method.

In [ ]:
galaxy.dump_to_h5()

PSF matching

In [ ]:
galaxy.add_psf_models()

Pixel Binning

In [ ]:
galaxy.pixel_by_pixel_galaxy_region(snr_req=3)

galaxy.pixel_by_pixel_binmap(galaxy_region = 'SNR_3_F444W', overwrite=True)

tab = galaxy.measure_flux_in_bins(binmap_type='pixel_by_pixel', overwrite=True)


Plotting overview

In [ ]:
galaxy.plot_overview(save=False, flux_unit = u.ABmag, show=True, binmap_type = 'pixel_by_pixel',
                    bins_to_show = ["TOTAL_BIN", "1"], 
                    bands_to_show = ['F606W', 'F115W', 'F200W', 'F356W', 'F444W'])

Interaction with piXedfit:

In [ ]:
galaxy.pixedfit_processing

galaxy.pisedfit_binning

SED Fitting with Bagpipes

In [ ]:
galaxy.run_bagpipes

Plotting of SED Results

In [ ]:
galaxy.plot_bagpipes_fit

galaxy.plot_bagpipes_sed

galaxy.plot_bagpipes_corner 

galaxy.plot_bagpipes_map_gif

galaxy.plot_bagpipes_component_comparison

galaxy.plot_bagpipes_sfh

SED Fitting with Dense Basis

In [ ]:
galaxy.run_dense_basis

Morphological Fitting with pysersic

In [ ]:
galaxy.run_pysersic

Morphological Fitting with GALFIT

In [ ]:
galaxy.run_galfitm

Morphological Fitting with pyautogalaxy

In [ ]:
galaxy.run_autogalaxy

EAZY SED Fitting

In [ ]:
galaxy.eazy_fit_measured_photometry

galaxy.plot_eazy_fit

Running SEXtractor for Python (SEP)

In [ ]:
galaxy.sep_process